# Looping over systematic models

Ideally, we would be using something from this in the docs to loop through our 50 different systematic models, which only differ in which parameters are thawed or frozen:  
https://sherpa.readthedocs.io/en/4.11.0/model_classes/api/sherpa.models.model.SimulFitModel.html#sherpa.models.model.SimulFitModel

Until such time that we understand how that works, we will be helping ourselves with a good old `for` loop. In this notebook, I am figuring out how.

In [ ]:
# Imports
import os
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u
from astropy.constants import G

os.chdir('../')
from config import CONFIG_INI
from limb_darkening import limb_dark_fit
import margmodule as marg

from sherpa.data import Data1D
from sherpa.plot import DataPlot
from sherpa.plot import ModelPlot
from sherpa.fit import Fit
from sherpa.stats import LeastSq
from sherpa.optmethods import LevMar
from sherpa.stats import Chi2
from sherpa.plot import FitPlot

### Data paths

In [ ]:
localDir = CONFIG_INI.get('data_paths', 'local_path')
outDir = os.path.join(localDir, CONFIG_INI.get('data_paths', 'output_path'))
curr_model = CONFIG_INI.get('data_paths', 'current_model')
dataDir = os.path.join(localDir, os.path.join(localDir, CONFIG_INI.get('data_paths', 'data_path')), curr_model)

### Read data

In [ ]:
# Read in the txt file for the lightcurve data
x, y, err, sh = np.loadtxt(os.path.join(dataDir, 'W17_white_lightcurve_test_data.txt'), skiprows=7, unpack=True)
wavelength = np.loadtxt(os.path.join(dataDir, 'W17_wavelength_test_data.txt'), skiprows=3)

tzero = x[0]
flux0 = y[0]

print("x.shape: {}".format(x.shape))
print("y.shape: {}".format(y.shape))
print("err.shape: {}".format(err.shape))
print("sh.shape: {}".format(sh.shape))
print("wvln.shape: {}".format(wavelength.shape))

### PLanet parameters and limb darkening

In [ ]:
Per = CONFIG_INI.getfloat('planet_parameters', 'Per') * u.d
Per = Per.to(u.s)
constant1 = ((G * np.square(Per)) / (4 * np.square(np.pi))) ** (1 / 3)
aor = CONFIG_INI.getfloat('planet_parameters', 'aor')    # this is unitless -> "distance of the planet from the star (meters)/stellar radius (meters)"
MsMpR = (aor / constant1) ** 3.
print("MsMpR: {}\n".format(MsMpR))

# Limb darkening
M_H = CONFIG_INI.getfloat('limb_darkening', 'metallicity')    # metallicity
Teff = CONFIG_INI.getfloat('limb_darkening', 'Teff')   # effective temperature
logg = CONFIG_INI.getfloat('limb_darkening', 'logg')   # log(g), gravitation

# Define limb darkening directory, which is inside this package
limbDir = os.path.join('..', 'Limb-darkening')
ld_model = CONFIG_INI.get('limb_darkening', 'ld_model')
grat = CONFIG_INI.get('technical_parameters', 'grating')
_uLD, c1, c2, c3, c4, _cp1, _cp2, _cp3, _cp4, _aLD, _bLD = limb_dark_fit(grat, wavelength, M_H,
                                                                         Teff, logg, limbDir, ld_model)
print("\nThe four cs: {}, {}, {}, {}".format(c1, c2, c3, c4))

### Sherpa data object

In [ ]:
# Instantiate a data object
data = Data1D('Data', x, y, staterror=err)
print(data)

# Plot the data with Sherpa
dplot = DataPlot()
dplot.prepare(data)
dplot.plot() 

### Sherpa model - here's where we start working

All parameters will be thawed, except for the ones that have `alwaysfrozen=True` in their initializaiton.

In [ ]:
# Define the model
tmodel = marg.Transit(tzero, MsMpR, c1, c2, c3, c4, flux0, name="testmodel", sh=sh)
print(tmodel)

We can read the requested grid from our module.

In [ ]:
grid_selection = CONFIG_INI.get('technical_parameters', 'grid_selection')
grid = marg.wfc3_systematic_model_grid_selection(grid_selection)
print('1st systematic model: {}'.format(grid[0]))

We can loop through that easily.

In [ ]:
for i, s in enumerate(grid):
    print(i+1, s)

So, `s` will be a single systematic model:

In [ ]:
print(s)

So far, so good.

Now, we can also loop though all the model parameters pretty easily.

In [ ]:
for i in tmodel.pars:
    print('->', i.name)
    print(i)
    print('\n')

Now we need to think of a way to combine `s` with the parameters in the model. Remember that `s` has as many entries as `tmodel` has parameters.

In [ ]:
# The first loop is the big loop that goes through all the systematic models.
# The second loop is what sets up the frozen and thawed parameters.
for i, s in enumerate(grid):
    for k, p in enumerate(s):
        print('i:', i)
        print('k:', p)

In [ ]:
# Now in our context with the model:

In [ ]:
for i, s in enumerate(grid):
    #print(s)
    for k, select in enumerate(s):
        if select == 0:
            tmodel.pars[k].thaw()
        elif select == 1:
            tmodel.pars[k].freeze()

How do we test this though? Lets do it one by one with the systematics.

In [ ]:
po = 36  # pick any between 0 and 49 (we have 50 systematic models)
sys = grid[po]
print(sys)

for k, select in enumerate(sys):
    if select == 0:
        tmodel.pars[k].thaw()
    elif select == 1:
        tmodel.pars[k].freeze()

print(tmodel)

Works!